In [1]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
import json

In [2]:
def getData(skipCount = 0, slug = "dien-thoai", maxResultCount = 16):
    response = requests.post("https://papi.fptshop.com.vn/gw/v1/public/fulltext-search-service/category", json={
        "skipCount": skipCount,
        "maxResultCount": maxResultCount,
        "sortMethod": "noi-bat",
        "slug": slug,
        "categoryType": "category"
    })
    if response.status_code != 200:
        return False

    return response.json()

def extract_data_from_script_with_keyword(url, keyword = "Thông tin hàng hóa"):
    # Fetch the webpage
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
        return None

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all <script> tags
    script_tags = soup.find_all('script')
    
    for script in script_tags:
        if keyword in script.text:
            try:
                data = script.text.split("self.__next_f.push(")[1].replace('\\n', '').replace('\\', '')
                data = f"[{{\"groupName\":\"{keyword}\"" + data[:len(data)-1].split(f'{{"groupName":"{keyword}"')[1].split('},"productAttributeHighlight"')[0]
                return json.loads(data) 
            except:
                pass
            
    return None

# Crawl Data for each category

In [13]:
def raw_to_table_data(raw_data, slug, keyword):
    data_fpt = []
    if raw_data:
        for item in raw_data["items"]:
            # Lặp qua từng sản phẩm trong kết quả
            list_item = item["skus"]

            link = "https://fptshop.com.vn/" + item["slug"]
            # đem này làm một bảng khác luôn đc á
            detail_info = extract_data_from_script_with_keyword(link, keyword)  
            
            brand = item["brand"]["name"] if "brand" in item else ""
            productType = item["productType"]["name"] if "productType" in item else ""
            group = item["group"]["name"] if "group" in item else ""
            
            for data in list_item:
                # Tạo dict chứa thông tin kiểu mẫu (variant) của sản phẩm
                variant_info = dict()
                for i in range(len(data["variants"])):
                    variant_info[data["variants"][i]["propertyName"]] = data["variants"][i]["displayValue"]

                # Tạo dict chứa thông tin sản phẩm
                data_fpt.append(
                    {
                        "link": link,
                        "detail_info": detail_info,

                        "sku": data["sku"] if "sku" in data else "",
                        "displayName": data["displayName"],
                        "brand": brand,
                        "productType": productType,
                        "group": group,
                        
                        "originalPrice": data["originalPrice"],
                        "currentPrice": data["currentPrice"],
                        "discountPercentage": data["discountPercentage"],
                        "endTimeDiscount": data["endTimeDiscount"],
                    }
                )

                # Bổ sung thông tin kiểu mẫu vào dict chứa thông tin sản phẩm
                data_fpt[-1].update(variant_info)
            time.sleep(1)

    return data_fpt

In [14]:
def crawl_data(category,keyword):
    data_fpt = []
    slug = category
    data = getData(0, slug)
    data_fpt = data_fpt + raw_to_table_data(data,slug,keyword)
    time.sleep(1)
    for i in range(16, data['totalCount'], 16):
        print(i)
        res = getData(i, slug)
        data_fpt = data_fpt + raw_to_table_data(res,slug,keyword)
    return data_fpt

# Test 1 sample


In [15]:
data_fpt = []
slug = "dien-thoai"
data = getData(0, slug, 1)
data_fpt = data_fpt + raw_to_table_data(data,slug,"Thông tin hàng hóa")
df = pd.DataFrame(data_fpt)

df.to_csv('test_3_sample.csv', index=True, header=True,encoding='utf-8-sig')

In [12]:
with open("data.json", "w") as f:
    json.dump(data, f)

# cào từng category và keyword tương ứng, cào tới đâu lưu tới đó tránh lỗi

In [16]:
category_list = ["dien-thoai", "may-tinh-xach-tay",'may-tinh-bang',"smartwatch","phu-kien",'tivi','may-giat','robot-hut-bui','may-tinh-de-ban','man-hinh','may-lanh-dieu-hoa','tu-lanh','may-loc-nuoc','quat-dieu-hoa',]#sim-fpt , linh-kien, dien-gia-dung, may-doi-tra
keyword_list = ["Thông tin hàng hóa","Bộ xử lý","Thông tin hàng hóa","Thông tin hàng hóa","Thông tin hàng hóa","Thông tin hàng hóa","Thông tin hàng hóa","Thông tin hàng hóa",'Bộ xử lý',"Thông tin hàng hóa","Thông tin hàng hóa","Thông tin hàng hóa","Thông tin hàng hóa","Thông tin hàng hóa"]

In [ ]:
print(len(category_list),len(keyword_list))
for i in range(len(keyword_list)):
    print(category_list[i],"\t",keyword_list[i])

In [ ]:
for i in range(len(category_list)):
    print(f'###Start crawling: {category_list[i]}')
    
    try:
        data = crawl_data(category_list[i],keyword_list[i])
        df = pd.DataFrame(data)
        df.to_csv(f'{category_list[i]}.csv',header = True, index = True,encoding='utf-8-sig')
    except:
        print(f'Error at {category_list[i]} , still continue to crawl')
        pass
    
    print(f'###Finish crawling: {category_list[i]}')

###Start crawling: dien-thoai
16
32
48


# struct of info_items

In [ ]:
############## struct of info_items
# [
#   {
#     "groupName": "Thông tin hàng hóa",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 117,
#         "propertyName": "Origin",
#         "displayName": "Xuất xứ",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "Trung Quốc"
#         ]
#       },
#       {
#         "id": 118,
#         "propertyName": "ReleaseTime",
#         "displayName": "Thời điểm ra mắt",
#         "isCompare": false,
#         "icon": null,
#         "value": "01/2024"
#       },
#       {
#         "id": 119,
#         "propertyName": "WarrantyMonth",
#         "displayName": "Thời gian bảo hành (tháng)",
#         "isCompare": false,
#         "icon": null,
#         "value": 13
#       },
#       {
#         "id": 121,
#         "propertyName": "PreserveGuide",
#         "displayName": "Hướng dẫn bảo quản",
#         "isCompare": false,
#         "icon": null,
#         "value": "Để nơi khô ráo, nhẹ tay, dễ vỡ."
#       },
#       {
#         "id": 120,
#         "propertyName": "UsageGuide",
#         "displayName": "Hướng dẫn sử dụng",
#         "isCompare": false,
#         "icon": null,
#         "value": "Xem trong sách hướng dẫn sử dụng"
#       }
#     ]
#   },
#   {
#     "groupName": "Thiết kế u0026 Trọng lượng",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 122,
#         "propertyName": "Size",
#         "displayName": "Kích thước",
#         "isCompare": false,
#         "icon": null,
#         "value": "166.7 x 76.5 x 8.24 mm"
#       },
#       {
#         "id": 123,
#         "propertyName": "ProductWeight",
#         "displayName": "Trọng lượng sản phẩm",
#         "isCompare": true,
#         "icon": null,
#         "value": [
#           {
#             "displayValue": "199 g",
#             "value": 199
#           }
#         ]
#       },
#       {
#         "id": 124,
#         "propertyName": "WaterDustResistantStandard",
#         "displayName": "Chuẩn kháng nước / Bụi bẩn",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 125,
#         "propertyName": "ScreenUsableAreaRatio",
#         "displayName": "Tỷ lệ diện tích sử dụng màn hình",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 126,
#         "propertyName": "Material",
#         "displayName": "Chất liệu",
#         "isCompare": true,
#         "icon": null,
#         "value": [
#           "Mặt lưng máy: Nhựa"
#         ]
#       }
#     ]
#   },
#   {
#     "groupName": "Bộ xử lý",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 127,
#         "propertyName": "cpuModel",
#         "displayName": "Phiên bản CPU",
#         "isCompare": true,
#         "icon": null,
#         "value": "Snapdragon 680"
#       },
#       {
#         "id": 128,
#         "propertyName": "cpuType",
#         "displayName": "Loại CPU",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 129,
#         "propertyName": "cpu",
#         "displayName": "CPU",
#         "isCompare": false,
#         "icon": null,
#         "value": "4 x A73 2.4 GHz + 4 x A53 1.9 GHz"
#       },
#       {
#         "id": 130,
#         "propertyName": "cpuCores",
#         "displayName": "Số nhân",
#         "isCompare": true,
#         "icon": null,
#         "value": 8
#       },
#       {
#         "id": 131,
#         "propertyName": "MaxSpeed",
#         "displayName": "Tốc độ tối đa",
#         "isCompare": false,
#         "icon": null,
#         "value": {
#           "displayValue": "2.4 GHz",
#           "value": 2.4
#         }
#       },
#       {
#         "id": 132,
#         "propertyName": "Is64Bits",
#         "displayName": "64 Bits",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       }
#     ]
#   },
#   {
#     "groupName": "RAM",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 133,
#         "propertyName": "ram",
#         "displayName": "RAM",
#         "isCompare": true,
#         "icon": null,
#         "value": "8 GB"
#       },
#       {
#         "id": 134,
#         "propertyName": "ramType",
#         "displayName": "Loại RAM",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       }
#     ]
#   },
#   {
#     "groupName": "Màn hình",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 135,
#         "propertyName": "ScreenSize",
#         "displayName": "Kích thước màn hình",
#         "isCompare": true,
#         "icon": null,
#         "value": {
#           "displayValue": "6.8 inch",
#           "value": 6.8
#         }
#       },
#       {
#         "id": 141,
#         "propertyName": "SubScreenSize",
#         "displayName": "Kích thước màn hình phụ",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 136,
#         "propertyName": "ScreenTechnology",
#         "displayName": "Công nghệ màn hình",
#         "isCompare": false,
#         "icon": null,
#         "value": "TFT LCD"
#       },
#       {
#         "id": 138,
#         "propertyName": "StandardScreen",
#         "displayName": "Chuẩn màn hình",
#         "isCompare": true,
#         "icon": null,
#         "value": "FHD+"
#       },
#       {
#         "id": 137,
#         "propertyName": "Resolution",
#         "displayName": "Độ phân giải",
#         "isCompare": true,
#         "icon": null,
#         "value": [
#           "1080 x 2412 Pixels"
#         ]
#       },
#       {
#         "id": 139,
#         "propertyName": "ScreenColor",
#         "displayName": "Màu màn hình",
#         "isCompare": false,
#         "icon": null,
#         "value": "16.7 Triệu"
#       },
#       {
#         "id": 140,
#         "propertyName": "RefreshRate",
#         "displayName": "Tần số quét",
#         "isCompare": false,
#         "icon": null,
#         "value": {
#           "displayValue": "90 ",
#           "value": 90
#         }
#       },
#       {
#         "id": 142,
#         "propertyName": "GlassMaterial",
#         "displayName": "Chất liệu mặt kính",
#         "isCompare": false,
#         "icon": null,
#         "value": "Kính cường lực"
#       },
#       {
#         "id": 143,
#         "propertyName": "TouchType",
#         "displayName": "Loại cảm ứng",
#         "isCompare": false,
#         "icon": null,
#         "value": "Cảm ứng đa điểm"
#       },
#       {
#         "id": 144,
#         "propertyName": "PixelDensity",
#         "displayName": "Mật độ điểm ảnh",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 145,
#         "propertyName": "Brightness",
#         "displayName": "Độ sáng",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 146,
#         "propertyName": "ColorGamut",
#         "displayName": "Độ phủ màu",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 147,
#         "propertyName": "ScreenRatio",
#         "displayName": "Tỷ lệ màn hình",
#         "isCompare": false,
#         "icon": null,
#         "value": "20:9"
#       },
#       {
#         "id": 148,
#         "propertyName": "Contrast",
#         "displayName": "Độ tương phản",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       }
#     ]
#   },
#   {
#     "groupName": "Đồ họa",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 149,
#         "propertyName": "gpu",
#         "displayName": "Chip đồ hoạ (GPU)",
#         "isCompare": false,
#         "icon": null,
#         "value": "Adreno 610"
#       }
#     ]
#   },
#   {
#     "groupName": "ANTUTU",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 150,
#         "propertyName": "AntutuBenchmark",
#         "displayName": "ANTUTU",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       }
#     ]
#   },
#   {
#     "groupName": "Lưu trữ",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 151,
#         "propertyName": "rom",
#         "displayName": "Dung lượng (ROM)",
#         "isCompare": true,
#         "icon": null,
#         "value": {
#           "displayValue": "256 GB",
#           "value": 256
#         }
#       },
#       {
#         "id": 152,
#         "propertyName": "AvailableMemory",
#         "displayName": "Bộ nhớ còn lại",
#         "isCompare": false,
#         "icon": null,
#         "value": "Khoảng 241 GB"
#       },
#       {
#         "id": 153,
#         "propertyName": "PhonebookCapacity",
#         "displayName": "Danh bạ lưu trữ",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 154,
#         "propertyName": "ExternalMemoryCard",
#         "displayName": "Thẻ nhớ ngoài",
#         "isCompare": false,
#         "icon": null,
#         "value": "MicroSD"
#       },
#       {
#         "id": 155,
#         "propertyName": "MaxMemoryCard",
#         "displayName": "Hỗ trợ thẻ nhớ tối đa",
#         "isCompare": false,
#         "icon": null,
#         "value": "1 TB"
#       }
#     ]
#   },
#   {
#     "groupName": "Camera sau",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 156,
#         "propertyName": "RearCameraQuantity",
#         "displayName": "Số camera sau",
#         "isCompare": false,
#         "icon": null,
#         "value": "Triple rear camera"
#       },
#       {
#         "id": 2783,
#         "propertyName": "CameraType1",
#         "displayName": "Camera",
#         "isCompare": false,
#         "icon": null,
#         "value": "Standard"
#       },
#       {
#         "id": 2784,
#         "propertyName": "Camera1Sensor",
#         "displayName": "Sensor",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2785,
#         "propertyName": "Camera1Resolution",
#         "displayName": "Resolution",
#         "isCompare": false,
#         "icon": null,
#         "value": "108.0 MP"
#       },
#       {
#         "id": 2786,
#         "propertyName": "TypeRearcamera1",
#         "displayName": "Type",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2787,
#         "propertyName": "Camera1Aperture",
#         "displayName": "Aperture",
#         "isCompare": false,
#         "icon": null,
#         "value": "ƒ/1.75"
#       },
#       {
#         "id": 2789,
#         "propertyName": "Camera1PixelSize",
#         "displayName": "Pixel size",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2788,
#         "propertyName": "iso",
#         "displayName": "ISO",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2790,
#         "propertyName": "CameraType2",
#         "displayName": "Camera",
#         "isCompare": false,
#         "icon": null,
#         "value": "Ultra Wide"
#       },
#       {
#         "id": 2791,
#         "propertyName": "Camera2Resolution",
#         "displayName": "Resolution",
#         "isCompare": false,
#         "icon": null,
#         "value": "5.0 MP"
#       },
#       {
#         "id": 2792,
#         "propertyName": "Camera2Sensor",
#         "displayName": "Sensor",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2793,
#         "propertyName": "TypeRearcamera2",
#         "displayName": "Type",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2794,
#         "propertyName": "Camera2Aperture",
#         "displayName": "Aperture",
#         "isCompare": false,
#         "icon": null,
#         "value": "ƒ/2.2"
#       },
#       {
#         "id": 2795,
#         "propertyName": "Camera2PixelSize",
#         "displayName": "Pixel Size",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2796,
#         "propertyName": "iso2",
#         "displayName": "ISO",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2797,
#         "propertyName": "CameraType3",
#         "displayName": "Camera",
#         "isCompare": false,
#         "icon": null,
#         "value": "Zoom"
#       },
#       {
#         "id": 2798,
#         "propertyName": "Camera3Resolution",
#         "displayName": "Resolution",
#         "isCompare": false,
#         "icon": null,
#         "value": "2.0 MP"
#       },
#       {
#         "id": 2799,
#         "propertyName": "Camera3Sensor",
#         "displayName": "Sensor",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2800,
#         "propertyName": "TypeRearcamera3",
#         "displayName": "Type",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2801,
#         "propertyName": "Camera3Aperture",
#         "displayName": "Aperture",
#         "isCompare": false,
#         "icon": null,
#         "value": "ƒ/2.4"
#       },
#       {
#         "id": 2802,
#         "propertyName": "Camera3PixelSize",
#         "displayName": "Pixel size",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2803,
#         "propertyName": "iso3",
#         "displayName": "ISO",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2804,
#         "propertyName": "CameraType4",
#         "displayName": "Camera",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2805,
#         "propertyName": "Camera4Resolution",
#         "displayName": "Resolution",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2806,
#         "propertyName": "Camera4Sensor",
#         "displayName": "Sensor",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2807,
#         "propertyName": "TypeRearCamera4",
#         "displayName": "Type",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2808,
#         "propertyName": "Camera4Aperture",
#         "displayName": "Aperture",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2809,
#         "propertyName": "Camera4PixelSize",
#         "displayName": "Pixel size",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2810,
#         "propertyName": "iso4",
#         "displayName": "ISO",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2811,
#         "propertyName": "CameraType5",
#         "displayName": "Camera",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2812,
#         "propertyName": "Camera5Resolution",
#         "displayName": "Resolution",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2813,
#         "propertyName": "Camera5Sensor",
#         "displayName": "Sensor",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2814,
#         "propertyName": "TypeRearCamera5",
#         "displayName": "Type",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2815,
#         "propertyName": "Camera5Aperture",
#         "displayName": "Aperture",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2816,
#         "propertyName": "Camera5PixelSize",
#         "displayName": "Pixel size",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2817,
#         "propertyName": "iso5",
#         "displayName": "ISO",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2818,
#         "propertyName": "FilmRearCamera",
#         "displayName": "Quay phim camera sau",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "1080p@30fps"
#         ]
#       },
#       {
#         "id": 2819,
#         "propertyName": "FeatureRearCamera",
#         "displayName": "Tính năng",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "Ban đêm (Night Mode)",
#           "Chụp chân dung",
#           "Chuyên nghiệp (Pro)",
#           "Toàn cảnh (Panorama)",
#           "HDR",
#           "Bộ lọc màu",
#           "Hình chìm mờ",
#           "Trôi nhanh thời gian (Time Lapse)",
#           "Chụp nụ cười",
#           "Quay chậm (Slow Motion)",
#           "Siêu cận (Macro)",
#           "Chế độ xem kép",
#           "Quét tài liệu"
#         ]
#       }
#     ]
#   },
#   {
#     "groupName": "Camera Selfie",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 194,
#         "propertyName": "SelfieCameraQuantity",
#         "displayName": "Số Camera Selfie",
#         "isCompare": false,
#         "icon": null,
#         "value": "Single selfie camera"
#       },
#       {
#         "id": 2820,
#         "propertyName": "SelfieCamera1Resolution",
#         "displayName": "Resolution",
#         "isCompare": false,
#         "icon": null,
#         "value": "8.0 MP"
#       },
#       {
#         "id": 2822,
#         "propertyName": "SelfieCamera1Sensor",
#         "displayName": "Sensor",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2821,
#         "propertyName": "SelfieCamera1Type",
#         "displayName": "Type",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2823,
#         "propertyName": "SelfieCamera1Aperture",
#         "displayName": "Aperture",
#         "isCompare": false,
#         "icon": null,
#         "value": "ƒ/2.0"
#       },
#       {
#         "id": 2824,
#         "propertyName": "SelfieCamera1PixelSize",
#         "displayName": "Pixel size",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2825,
#         "propertyName": "ISOCameraSelfie1",
#         "displayName": "ISO",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2826,
#         "propertyName": "ResolutionCameraSelfie2",
#         "displayName": "Resolution",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2827,
#         "propertyName": "SensorCameraSelfie2",
#         "displayName": "Sensor",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2828,
#         "propertyName": "TypeCameraSelfie2",
#         "displayName": "Type",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2829,
#         "propertyName": "Aperture",
#         "displayName": "Aperture",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2830,
#         "propertyName": "Pixelsize",
#         "displayName": "Pixel size",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2831,
#         "propertyName": "ISOCameraSelfie2",
#         "displayName": "ISO",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2832,
#         "propertyName": "ResolutionCameraSelfie3",
#         "displayName": "Resolution",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2833,
#         "propertyName": "SensorCameraSelfie3",
#         "displayName": "Sensor",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2834,
#         "propertyName": "TypeCameraSelfie3",
#         "displayName": "Type",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2835,
#         "propertyName": "ApertureCameraSelfie3",
#         "displayName": "Aperture",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2836,
#         "propertyName": "PixelsizeCameraSelfie3",
#         "displayName": "Pixel size",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2837,
#         "propertyName": "ISOCameraSelfie3",
#         "displayName": "ISO",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2838,
#         "propertyName": "FilmSelfieCamera",
#         "displayName": "Quay phim camera selfie",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 2839,
#         "propertyName": "FeatureSelfieCamera",
#         "displayName": "Tính năng",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "Chế độ chân dung",
#           "Bộ lọc màu",
#           "Chụp nụ cười",
#           "Gương phản chiếu",
#           "Hẹn giờ chụp tự động",
#           "Điều khiển bằng động tác"
#         ]
#       }
#     ]
#   },
#   {
#     "groupName": "Cảm biến",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 215,
#         "propertyName": "Sensor",
#         "displayName": "Cảm biến",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       }
#     ]
#   },
#   {
#     "groupName": "Bảo mật",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 216,
#         "propertyName": "Security",
#         "displayName": "Bảo mật",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       }
#     ]
#   },
#   {
#     "groupName": "Others",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 219,
#         "propertyName": "Cooling",
#         "displayName": "Làm mát",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 217,
#         "propertyName": "ledFlashAlert",
#         "displayName": "Thông báo LED",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 218,
#         "propertyName": "Feature",
#         "displayName": "Tính năng đặc biệt",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       }
#     ]
#   },
#   {
#     "groupName": "Giao tiếp và kết nối",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 3324,
#         "propertyName": "simTray",
#         "displayName": "Số khe SIM",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 3325,
#         "propertyName": "simType",
#         "displayName": "Thẻ SIM",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 222,
#         "propertyName": "NetworkSupported",
#         "displayName": "Hỗ trợ mạng",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "4G",
#           "3G"
#         ]
#       },
#       {
#         "id": 223,
#         "propertyName": "SerialPort",
#         "displayName": "Cổng giao tiếp",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       },
#       {
#         "id": 224,
#         "propertyName": "Wifi",
#         "displayName": "Wifi",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "802.11 a/b/g/n/ac"
#         ]
#       },
#       {
#         "id": 225,
#         "propertyName": "GPS",
#         "displayName": "GPS",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "GPS",
#           "A-GPS",
#           "GLONASS",
#           "BEIDOU",
#           "GALILEO"
#         ]
#       },
#       {
#         "id": 226,
#         "propertyName": "Bluetooth",
#         "displayName": "Bluetooth",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "v5.0"
#         ]
#       },
#       {
#         "id": 227,
#         "propertyName": "OtherConnection",
#         "displayName": "Kết nối khác",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       }
#     ]
#   },
#   {
#     "groupName": "Thông tin pin u0026 sạc",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 228,
#         "propertyName": "pinType",
#         "displayName": "Loại PIN",
#         "isCompare": false,
#         "icon": null,
#         "value": "Lithium polymer"
#       },
#       {
#         "id": 229,
#         "propertyName": "BatteryCapacity",
#         "displayName": "Dung lượng pin",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           {
#             "displayValue": "6000 mAh",
#             "value": 6000
#           }
#         ]
#       },
#       {
#         "id": 5005,
#         "propertyName": "IsChargerIncluded",
#         "displayName": "Củ sạc kèm máy",
#         "isCompare": false,
#         "icon": null,
#         "value": "Sạc nhanh 35 W"
#       },
#       {
#         "id": 5002,
#         "propertyName": "OtherInformation",
#         "displayName": "Thông tin thêm",
#         "isCompare": false,
#         "icon": null,
#         "value": null
#       }
#     ]
#   },
#   {
#     "groupName": "Hệ điều hành",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 230,
#         "propertyName": "os",
#         "displayName": "OS",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "Android"
#         ]
#       },
#       {
#         "id": 231,
#         "propertyName": "Version",
#         "displayName": "Version",
#         "isCompare": false,
#         "icon": null,
#         "value": "Android 13.0"
#       }
#     ]
#   },
#   {
#     "groupName": "Phụ kiện trong hộp",
#     "groupIcon": null,
#     "attributes": [
#       {
#         "id": 232,
#         "propertyName": "AccessoriesInsideBox",
#         "displayName": "Phụ kiện trong hộp",
#         "isCompare": false,
#         "icon": null,
#         "value": [
#           "Ốp lưng",
#           "Sạc",
#           "Sách HDSD"
#         ]
#       }
#     ]
#   }
# ]